## The problem

Topic modeling is a fun way to start our study of NLP. We will use two popular **matrix decomposition techniques**.

We start with a **term-document matrix**:

![[Pasted image 20220607090648.png | 750]]

We can decompose this into one tall thin matrix times one wide short matrix (possibly with a diagnoal matrix in between).

### Motivation

Consider the most extreme case

## Getting started

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn import decomposition
from scipy import linalg
import matplotlib.pyplot as plt

%matplotlib inline
np.set_printoptions(suppress=True)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)
newsgroups_train.filenames.shape, newsgroups_train.target.shape

In [ ]:
print("\n".join(newsgroups_train.data[:3]))

In [ ]:
np.array(newsgroups_train.target_names)[newsgroups_train.target[:3]]

In [ ]:
newsgroups_train.target[:10]

In [ ]:
num_topics, num_top_words = 6, 8

## Stop words, stemming, lemmatization

### Stop words

From Intro to Information Retrieval:

Some extremely common words which would appear to be of little value in helping select documents matching a user need are excluded from the vocabulary entirely. These words are called stop words.

The general trend in IR systems over time has been from standard use of quite large stop lists (200-300 terms) to very small stop lists (7-12 terms) to no stop list whatsoever. Web search engines generally do not use stop lists.

### NLTK

In [ ]:
from sklearn.feature_extraction import stop_words

sorted(list(stop_words.ENGLISH_STOP_WORDS))[:20]